# Download the .nc files and save in a seperate folder.

In [56]:
import os
import requests

def download_files_from_links(filename):
    folder_name = os.path.splitext(filename)[0]  # Extract the folder name from the text file name
    os.makedirs(folder_name, exist_ok=True)  # Create the folder if it doesn't exist

    with open(filename, 'r') as file:
        links = file.readlines()

    for link in links:
        link = link.strip()
        file_name = link.split('/')[-1].split('?')[0]  # Extract the file name from the URL
        file_path = os.path.join(folder_name, file_name)  # Create the file path within the folder

        print(f"Downloading {file_name}...")

        response = requests.get(link)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"{file_name} downloaded successfully.")
        else:
            print(f"Failed to download {file_name}.")

# Provide the filename of the text file containing the links
filename = "/Users/jennifergao/ClimateData_NT/S202306181753294947802.txt"
download_files_from_links(filename)


DQ1ch1TK7wS5lr5jyYtwYik%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
Z_NAFP_C_BABJ_20210109000608_P_CLDAS_NRT_ASI_0P0625_DAY-PRE-2021010600.nc downloaded successfully.
cPsT0%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
Z_NAFP_C_BABJ_20210110000616_P_CLDAS_NRT_ASI_0P0625_DAY-PRE-2021010700.nc downloaded successfully.


# Download Multiple text files

In [101]:
import os
import requests

def download_files_from_links(filename_prefix, start_number, end_number):
    for i in range(start_number, end_number + 1):
        # Generate the filename with changing numbers
        filename = f"{filename_prefix}{i:03}.txt"

        folder_name = os.path.splitext(filename)[0]  # Extract the folder name from the text file name
        os.makedirs(folder_name, exist_ok=True)  # Create the folder if it doesn't exist

        with open(filename, 'r') as file:
            links = file.readlines()

        for link in links:
            link = link.strip()
            file_name = link.split('/')[-1].split('?')[0]  # Extract the file name from the URL
            file_path = os.path.join(folder_name, file_name)  # Create the file path within the folder

            print(f"Downloading {file_name}...")

            response = requests.get(link)
            if response.status_code == 200:
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"{file_name} downloaded successfully.")
            else:
                print(f"Failed to download {file_name}.")

# Provide the filename prefix and the range of file numbers
filename_prefix = "/Users/jennifergao/ClimateData_NT/S202306182355084957"
start_number = 400
end_number = 430
download_files_from_links(filename_prefix, start_number, end_number)


Z_NAFP_C_BABJ_20210403060021_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040106.nc downloaded successfully.
Z_NAFP_C_BABJ_20210403060055_P_CLDAS_NRT_ASI_0P0625_HOR-TMP-2021040106.nc downloaded successfully.
Z_NAFP_C_BABJ_20210403070024_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040107.nc downloaded successfully.
Z_NAFP_C_BABJ_20210403070030_P_CLDAS_NRT_ASI_0P0625_HOR-TMP-2021040107.nc downloaded successfully.
3bzdZX7qU%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
%2BW3TbvntTyMr6YmwOrrWdLWo%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
bw%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
QdYOaVtikGkprb3xw%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
Z_NAFP_C_BABJ_20210403100024_P_CLDAS_NRT_ASI_0P0625_HOR-TMP-2021040110.nc downloaded successfully.
Z_NAFP_C_BABJ_20210403100016_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040

# read the nc file

In [64]:
import os
import netCDF4 as nc
import openpyxl as xl
import numpy as np

# Set the folder path
folder_path = '/Users/jennifergao/ClimateData_NT/S202306181753294947802'

# Get all nc files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.nc')]

# Create an empty list to store the data
data_list = []

# Create an Excel file
excel_file = '1003-1005.xlsx'
if os.path.exists(excel_file):
    os.remove(excel_file)

# Loop through each nc file
for file_name in file_list:
    # Read the nc file
    file_path = os.path.join(folder_path, file_name)
    try:
        nc_info = nc.Dataset(file_path)
        variable_keys = list(nc_info.variables.keys())
        var_data = np.array(nc_info.variables[variable_keys[-1]][:])  # Convert the data to a numpy array
        var_data_flattened = var_data.flatten().tolist()  # Flatten the data and convert to a list
        data_list.append([file_name, variable_keys[-1], var_data_flattened])
        nc_info.close()
    except OSError as e:
        print(f"OSError: {e}")
        print(f"Could not read the file: {file_path}. It may not have been properly converted. Skipping this file.")

# Specify the filename and sheet name for the Excel file
filename = '1003-1005.xlsx'
sheetname = 'Sheet1'

# Write the data to the Excel file
workbook = xl.Workbook()
sheet = workbook.active
for row in data_list:
    cleaned_row = []
    for value in row:
        if isinstance(value, list):
            # Flatten the nested list and replace large values with a placeholder
            cleaned_sublist = [sub_value if (isinstance(sub_value, str) or abs(sub_value) < 1e35) else "N/A" for sub_value in value]
            cleaned_row.extend(cleaned_sublist)
        else:
            # Replace large values with a placeholder
            cleaned_value = value if (isinstance(value, str) or abs(value) < 1e35) else "N/A"
            cleaned_row.append(cleaned_value)
    sheet.append(cleaned_row)
workbook.save(filename)


In [102]:
# Multiple .nc files in multiple folders

import os
import netCDF4 as nc
import openpyxl as xl
import numpy as np

# Set the folder path prefix and the range of folder numbers
folder_path_prefix = '/Users/jennifergao/ClimateData_NT/S202306182355084957'
start_number = 400
end_number = 430

# Create an empty list to store the data
data_list = []

# Create an Excel file
excel_file = '1003-1005.xlsx'
if os.path.exists(excel_file):
    os.remove(excel_file)

# Loop through each folder
for i in range(start_number, end_number + 1):
    folder_path = f"{folder_path_prefix}{i:03}"
    
    # Get all nc files in the folder
    file_list = [file for file in os.listdir(folder_path) if file.endswith('.nc')]
    
    # Loop through each nc file
    for file_name in file_list:
        # Read the nc file
        file_path = os.path.join(folder_path, file_name)
        try:
            nc_info = nc.Dataset(file_path)
            variable_keys = list(nc_info.variables.keys())
            var_data = np.array(nc_info.variables[variable_keys[-1]][:])  # Convert the data to a numpy array
            var_data_flattened = var_data.flatten().tolist()  # Flatten the data and convert to a list
            data_list.append([file_name, variable_keys[-1], var_data_flattened])
            nc_info.close()
        except OSError as e:
            print(f"OSError: {e}")
            print(f"Could not read the file: {file_path}. It may not have been properly converted. Skipping this file.")

# Specify the filename and sheet name for the Excel file
filename = '1003-1005.xlsx'
sheetname = 'Sheet1'

# Write the data to the Excel file
workbook = xl.Workbook()
sheet = workbook.active
for row in data_list:
    cleaned_row = []
    for value in row:
        if isinstance(value, list):
            # Flatten the nested list and replace large values with a placeholder
            cleaned_sublist = [sub_value if (isinstance(sub_value, str) or abs(sub_value) < 1e35) else "N/A" for sub_value in value]
            cleaned_row.extend(cleaned_sublist)
        else:
            # Replace large values with a placeholder
            cleaned_value = value if (isinstance(value, str) or abs(value) < 1e35) else "N/A"
            cleaned_row.append(cleaned_value)
    sheet.append(cleaned_row)
workbook.save(filename)


In [103]:
import openpyxl as xl

# Set the Excel filename and data start row
filename = '1003-1005.xlsx'
data_start_row = 1

# Load the data from the Excel file
workbook = xl.load_workbook(filename)
sheet = workbook.active
raw_data = list(sheet.iter_rows(values_only=True))[data_start_row-1:]

# Process each row of data
variable_meaning = []
data = []
for row in raw_data:
    file_name, variable_name, variable_value = row[:3]
    
    # Add a check here
    if not isinstance(file_name, str):
        print(f"Unexpected data type for file_name: {file_name} ({type(file_name)})")
        continue

    datetime = file_name[-13:-5]  # Extract the last 10 characters from the file name


    # Check the variable name and carry out different operations depending on its value
    if variable_name == 'PRCP_DAY_SUM':
        variable_meaning.append('1小时相对降水')
    elif variable_name == 'PAIR_DAY_AVG':
        variable_meaning.append('地面气压')
    elif variable_name == 'QAIR_DAY_AVG':
        variable_meaning.append('比湿')
    elif variable_name == 'SWDN_DAY_AVG':
        variable_meaning.append('短波辐射')
    elif variable_name == 'TAIR_DAY_AVG':
        variable_meaning.append('2m气温')
    elif variable_name == 'WIND_DAY_AVG':
        variable_meaning.append('10m风速')
    elif variable_name == 'TG_DAY_MIN':
        variable_meaning.append('地表温度')
    elif variable_name == 'SOILLIQ':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'SM000005':
            variable_meaning.append('0-5cm土壤湿度')
        elif var_name == 'SM000010':
            variable_meaning.append('0-10cm土壤湿度')
        elif var_name == 'SM010040':
            variable_meaning.append('10-40cm土壤湿度')
        elif var_name == 'SM040100':
            variable_meaning.append('40-100cm土壤湿度')
        elif var_name == 'SM100200':
            variable_meaning.append('100-200cm土壤湿度')
        else:
            variable_meaning.append('')
    elif variable_name == 'TSOI':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'GST005':
            variable_meaning.append('5cm土壤温度')
        elif var_name == 'GST010':
            variable_meaning.append('10cm土壤温度')
        elif var_name == 'GST040':
            variable_meaning.append('40cm土壤温度')
        elif var_name == 'GST100':
            variable_meaning.append('100cm土壤温度')
        elif var_name == 'GST200':
            variable_meaning.append('200cm土壤温度')
    elif variable_name == 'RSM':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'RSM000010':
            variable_meaning.append('0-10cm土壤相对湿度')
        elif var_name == 'RSM000020':
            variable_meaning.append('0-20cm土壤相对湿度')
        elif var_name == 'RSM000050':
            variable_meaning.append('0-50cm土壤相对湿度')
        else:
            variable_meaning.append('')
    else:
        variable_meaning.append('')

    # Add the file name, datetime, variable name, variable value and variable meaning to the data list
    data.append([file_name, datetime, variable_name, variable_value, variable_meaning[-1]])

# Write the data to the Excel file
output_sheet = workbook.create_sheet(title='Sheet2')
output_sheet.append(['File Name', 'Datetime', 'Variable Name', 'Variable Value', 'Variable Meaning'])
for row in data:
    output_sheet.append(row)

# Save the Excel file
workbook.save(filename)

# Display the results
print('Data with variable meanings:')
for row in data:
    print(row)


Data with variable meanings:
['Z_NAFP_C_BABJ_20210404230018_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040223.nc', '20210402', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210405020022_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040302.nc', '20210403', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210404040044_P_CLDAS_NRT_ASI_0P0625_HOR-TMP-2021040204.nc', '20210402', 'TAIR', 286.9185791015625, '']
['Z_NAFP_C_BABJ_20210403070024_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040107.nc', '20210401', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210405010028_P_CLDAS_NRT_ASI_0P0625_HOR-TMP-2021040301.nc', '20210403', 'TAIR', 285.2958374023438, '']
['Z_NAFP_C_BABJ_20210405000026_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040300.nc', '20210403', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210405050017_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040305.nc', '20210403', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210404210023_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040221.nc', '20210402', 'PRCP', 0, '']
['Z_NAFP_C_BABJ_20210404110019_P_CLDAS_NRT_ASI_0P0625_HOR-PRE-2021040211.nc', '20210402', 'PRCP', 0